Adapted from `rlcard/examples/leduc_holdem_dqn_pytorch.py`

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('./rlcard'))

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import torch
import rlcard
import numpy as np
from DQNAgent import DQNAgent
from rlcard.agents import RandomAgent, CFRAgent
from rlcard.utils import set_global_seed, tournament
from rlcard.utils import Logger

In [4]:
set_global_seed(0)

In [5]:
eval_env = rlcard.make('leduc-holdem', config={'seed': 0})

In [6]:
actions = ['call', 'raise', 'fold', 'check']

In [7]:
dqn_agent = DQNAgent(
    eval_env.action_num,
    eval_env.state_shape[0],
    hidden_neurons=[64]
)

dqn_agent.load(os.path.join('models/leduc_holdem_dqn', 'model.pth'))

Indices 0-2 represent the hero's hole card
* 0 - Jack
* 1 - Queen
* 2 - King

Indices 3-5 represent the public card
* 3 - Jack
* 4 - Queen
* 5 - King

Indices 6-20 represent the number of chips the hero has bet
* 6 + num chips

Indices 21-35 represent the number of chips the villain has bet
* 21 + num chips

In [8]:
def get_obs(hole_card, public_card, hero_bet, villain_bet):
    obs = np.zeros(36)
    card_to_index = { 'J': 0, 'Q': 1, 'K': 2 }
    obs[card_to_index[hole_card]] = 1
    
    if public_card:
        obs[card_to_index[public_card] + 3] = 1
    
    obs[hero_bet + 6] = 1
    obs[villain_bet + 21] = 1
    return obs

def get_actions(moves):
    return [actions.index(a) for a in moves]

## First round

### Jack gole card

First to act or facing check

In [94]:
state = { 'obs': get_obs('J', '', 1, 1), 'legal_actions': get_actions(['raise', 'fold', 'check']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

Facing bet

In [95]:
state = { 'obs': get_obs('J', '', 1, 3), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

### Queen hole card

First to act or facing check

In [96]:
state = { 'obs': get_obs('Q', '', 1, 1), 'legal_actions': get_actions(['raise', 'fold', 'check']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

Facing bet

In [97]:
state = { 'obs': get_obs('Q', '', 1, 3), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

### King hole card

First to act or facing check

In [98]:
state = { 'obs': get_obs('K', '', 1, 1), 'legal_actions': get_actions(['raise', 'fold', 'check']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'raise'

Facing bet

In [99]:
state = { 'obs': get_obs('K', '', 1, 3), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'raise'

Bet and then facing raise

In [100]:
state = { 'obs': get_obs('K', '', 3, 5), 'legal_actions': get_actions(['call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

## Second round

### Jack hole card

#### Jack public card

- First round: check-check
- Second round: first to act or facing check

In [101]:
state = { 'obs': get_obs('J', 'J', 1, 1), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: check-check
- Second round: facing bet

In [102]:
state = { 'obs': get_obs('J', 'J', 1, 5), 'legal_actions': get_actions(['raise', 'call', 'fold']) }

index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

#### Queen public card

- First round: check-check
- Second round: first to act or facing check

In [103]:
state = { 'obs': get_obs('J', 'Q', 1, 1), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: check-check
- Second round: facing bet

In [105]:
state = { 'obs': get_obs('J', 'Q', 1, 5), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

#### King public card

- First round: check-check
- Second round: first to act or facing check

In [106]:
state = { 'obs': get_obs('J', 'K', 1, 1), 'legal_actions': get_actions(['raise', 'check', 'fold'])}
index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

- First round: check-check
- Second round: facing bet

In [107]:
state = { 'obs': get_obs('J', 'K', 1, 5), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

### Queen hole card

#### Jack public card

- First round: check-check
- Second round: first to act or facing check

In [109]:
state = { 'obs': get_obs('Q', 'J', 1, 1), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: check-check
- Second round: facing bet

In [110]:
state = { 'obs': get_obs('Q', 'J', 1, 5), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

- First round: called bet
- Second round: first to act or facing check

In [111]:
state = { 'obs': get_obs('Q', 'J', 3, 3), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: called bet
- Second round: facing bet

In [93]:
state = { 'obs': get_obs('Q', 'J', 3, 5), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

#### Queen public card

- First round: check-check
- Second round: first to act or facing check

In [92]:
state = { 'obs': get_obs('Q', 'Q', 1, 1), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: check-check
- Second round: facing bet

In [91]:
state = { 'obs': get_obs('Q', 'Q', 1, 5), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

- First round: called bet
- Second round: first to act or facing check

In [90]:
state = { 'obs': get_obs('Q', 'Q', 3, 3), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'raise'

- First round: called bet
- Second round: facing bet

In [89]:
state = { 'obs': get_obs('Q', 'Q', 3, 7), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

#### King public card

- First round: check-check
- Second round: first to act or facing check

In [88]:
state = { 'obs': get_obs('Q', 'K', 1, 1), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: check-check
- Second round: facing bet

In [87]:
state = { 'obs': get_obs('Q', 'K', 1, 5), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

- First round: called bet
- Second round: first to act or facing check

In [86]:
state = { 'obs': get_obs('Q', 'K', 3, 3), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: called bet
- Second round: facing bet

In [85]:
state = { 'obs': get_obs('Q', 'K', 3, 7), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

### King hole card

#### Jack public card

- First round: called bet
- Second round: first to act or facing check

In [83]:
state = { 'obs': get_obs('K', 'J', 3, 3), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: called bet
- Second round: facing bet

In [82]:
state = { 'obs': get_obs('K', 'J', 3, 7), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'fold'

- First round: called raise
- Second round: first to act or facing check

In [81]:
state = { 'obs': get_obs('K', 'J', 5, 5), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: called raise
- Second round: facing bet

In [80]:
state = { 'obs': get_obs('K', 'J', 5, 9), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

#### Queen public card

- First round: called bet
- Second round: first to act or facing check

In [79]:
state = { 'obs': get_obs('K', 'Q', 3, 3), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: called bet
- Second round: facing bet

In [75]:
state = { 'obs': get_obs('K', 'Q', 3, 7), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

- First round: called raise
- Second round: first to act or facing check

In [76]:
state = { 'obs': get_obs('K', 'Q', 5, 5), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'check'

- First round: called raise
- Second round: facing bet

In [77]:
state = { 'obs': get_obs('K', 'Q', 5, 9), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'call'

#### King public card

- First round: called bet
- Second round: first to act or facing check

In [70]:
state = { 'obs': get_obs('K', 'K', 3, 3), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'raise'

- First round: called bet
- Second round: facing bet

In [78]:
state = { 'obs': get_obs('K', 'K', 3, 7), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'raise'

- First round: called raise
- Second round: first to act or facing check

In [68]:
state = { 'obs': get_obs('K', 'K', 5, 5), 'legal_actions': get_actions(['raise', 'check', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'raise'

- First round: called raise
- Second round: facing bet

In [66]:
state = { 'obs': get_obs('K', 'K', 5, 9), 'legal_actions': get_actions(['raise', 'call', 'fold']) }
index, _ = dqn_agent.eval_step(state)
actions[index]

'call'